<a href="https://colab.research.google.com/github/jackychia/AIforCE/blob/master/yahoo_regression_of_%E5%9C%8B%E6%B3%B0%E9%87%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 交通土木108下_人工智慧

---
組員：


1.   黃柏瑜 0511328
2.   賈鎮宇 0511318













# Basic regression : Predict 國泰金

##  < setup background

In [ ]:
!pip install seaborn
!pip install git+https://github.com/tensorflow/docs

In [ ]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

## < dataset Processing

### < < Get the data

Import it using pandas

In [ ]:
#import檔案

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" to id:fn  (with length {length} bytes)'.format(name=fn, length=len(uploaded[fn])))
#column_names = ['open_pre','open','high','low','close','adj_close','volume']
dataset = pd.read_csv(fn,na_values = "0")

In [ ]:
dataset

### Make prediction columns

In [ ]:
dataset['Open_pre'] = dataset['Open'].shift(-1)

### < <  Clean the data

The dataset contains a few unknown values.

In [ ]:
dataset.isna().sum()
dataset = dataset.dropna()

In [ ]:
dataset

### < < Normalize the data

Look again at the `train_stats` block above and note how different the ranges of each feature are.

In [ ]:
def norm(x):
  A = x.pop('Date')  #從x當中drop out資料Date,並傳給A
  x = (x - dataset.min()) / (dataset.max() - dataset.min())
  x = x.transpose()
  x = x.append(A)
  x = x.transpose()
  return x


normed_dataset = norm(dataset)
#dataset=x dataset被drop掉Date的資料

In [ ]:
normed_dataset

This normalized data is what we will use to train the model.

注意：未來輸入其他資料也需要normalize

### < < Split the data into train and test

Now split the dataset into a training set and a test set.

We will use the test set in the final evaluation of our model.

In [ ]:
# `random_state` for reproducibility

normed_train_dataset = normed_dataset.sample(frac=0.8,random_state=2)
normed_test_dataset = normed_dataset.drop(normed_train_dataset.index)

In [ ]:
normed_train_dataset

In [ ]:
normed_test_dataset

### < < Inspect the data

Have a quick look at the joint distribution of a all pairs of columns from the training set.

In [ ]:
sns.pairplot(normed_train_dataset[["Open_pre","Open","High","Low","Close","Adj Close","Volume"]], diag_kind="kde")

Also look at the overall statistics:

In [ ]:
dataset

In [ ]:
B=dataset
normed_train_stats_B = B.describe()
normed_train_stats_B = normed_train_stats_B.transpose()
normed_train_stats_B

### < < Split features from labels

Separate the target value, or "label", from the features. 

This label is the value that you will train the model to predict.

In [ ]:
normed_train_labels = normed_train_dataset.pop('Open_pre')  #將Open_pre預測值開盤從normed_train_dataset中提出,並提給normed_train_labels
normed_test_labels = normed_test_dataset.pop('Open_pre')

In [ ]:
normed_train_labels

In [ ]:
normed_test_labels

## < The model

### < < Build the model

In [ ]:
def train_windows(df, ref_day=5, predict_day=1):
    normed_train_dataset, normed_train_labels = [], []
    for i in range(df.shape[0]-predict_day-ref_day):
        X_train_labels.append(np.array(df.iloc[i:i+ref_day,:-1]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]["open_pre"]))
    return np.array(normed_train_dataset), np.array(normed_train_labels)

def build_model():
    model = keras.Sequential()
    model.add(layers.LSTM(256, input_shape=(shape[1], shape[2]), return_sequences=True))
    model.add(layers.BatchNormalization())
    model.add(layers.LSTM(256, return_sequences=True))
    model.add(layers.BatchNormalization())
    model.add(layers.TimeDistributed(Dense(1)))
    model.add(layers.Flatten())
    model.add(layers.Dense(5,activation='linear'))
    model.add(layers.Dense(1,activation='linear'))

In [ ]:
#def build_model():
#  model = keras.Sequential()
#  model.add(layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]))
#  model.add(layers.BatchNormalization())
#  model.add(layers.Dense(64, activation='relu'))
#  model.add(layers.BatchNormalization())
#  model.add(layers.Dense(1))

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

### < < Inspect the model

Use the `.summary` method to print a simple description of the model

In [ ]:
model.summary()

Now try out the model. Take a batch of `10` examples from the training data and call `model.predict` on it.

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

It seems to be working, and it produces a result of the expected shape and type.

### < < Train the model

Train the model for 1000 epochs, and record the training and validation accuracy in the `history` object.

In [ ]:
EPOCHS = 1000
Batch_size = 32
batch_size=eval(input('The batch_size (default= 32):'))
epochs=eval(input('Epochs (default 1000):' ))

history = model.fit(
    normed_train_data, train_labels,
    batch_size=Batch_size,epochs=EPOCHS,
    validation_split = 0.1, verbose=0,
    callbacks=[tfdocs.modeling.EpochDots()])

Visualize the model's training progress using the stats stored in the `history` object.

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

In [ ]:
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [MPG]')

In [ ]:
plotter.plot({'Basic': history}, metric = "mse")
plt.ylim([0, 20])
plt.ylabel('MSE [MPG^2]')

This graph shows little improvement, or even degradation in the validation error after about 100 epochs. Let's update the `model.fit` call to automatically stop training when the validation score doesn't improve. We'll use an *EarlyStopping callback* that tests a training condition for  every epoch. If a set amount of epochs elapses without showing improvement, then automatically stop the training.

You can learn more about this callback [here](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping).

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

In [ ]:
plotter.plot({'Early Stopping': early_history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [MPG]')

The graph shows that on the validation set, the average error is usually around +/- 2 MPG. Is this good? We'll leave that decision up to you.

Let's see how well the model generalizes by using the **test** set, which we did not use when training the model.  This tells us how well we can expect the model to predict when we use it in the real world.

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

### < < Make predictions

Finally, predict MPG values using data in the testing set:

In [ ]:
print("Number of weights after calling the model:", model.weights)

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [32, 47]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


It looks like our model predicts reasonably well. Let's take a look at the error distribution.

In [ ]:
uploaded = files.upload()
for single_test in uploaded.keys():
  print('User uploaded file "{name}" to id:single_test  (with length {length} bytes)'.format(name=single_test, length=len(uploaded[single_test])))

In [ ]:
column_names1 = ['open','high','low','close','adj_close','volume']
X_test = pd.read_csv(single_test,names=column_names1,na_values = "0")
X_test

In [ ]:
X_test=norm(X_test)
X_test

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")

It's not quite gaussian, but we might expect that because the number of samples is very small.

## < Conclusion

This notebook introduced a few techniques to handle a regression problem.

* Mean Squared Error (MSE) is a common loss function used for regression problems (different loss functions are used for classification problems).
* Similarly, evaluation metrics used for regression differ from classification. A common regression metric is Mean Absolute Error (MAE).
* When numeric input data features have values with different ranges, each feature should be scaled independently to the same range.
* If there is not much training data, one technique is to prefer a small network with few hidden layers to avoid overfitting.
* Early stopping is a useful technique to prevent overfitting.